In [1]:
import pymongo, csv
import pandas as pd
import numpy as np
from pymongo import MongoClient
client = MongoClient()
pd.options.display.max_rows = 999

In [2]:
grade_map_path = '/Users/colinbrochard/DSI_Capstone_local/MtProjRec/6_app/grade_map.csv'
grade_num_map_path = '/Users/colinbrochard/DSI_Capstone_local/MtProjRec/6_app/grade_num_map.csv'
write_loc = '/Users/colinbrochard/DSI_Capstone_local/MtProjRec/2_data/5_route_features/routes_features.csv'

In [3]:
# pull data from Mongo DB into pd DF
db = client.routesAPI_db
df = pd.DataFrame(list(db.routesAPI_db.find()))

In [4]:
# clean up ratings
with open(grade_map_path) as f:
    data = [line.strip().split("|") for line in f.readlines()]
grade_map = {d[0]: d[1] for d in data}
df['rating0'] = df['rating'].map(grade_map)

In [5]:
# map ratings to number
with open(grade_num_map_path) as f:
    data = [line.strip().split("|") for line in f.readlines()]
grade_num_map = {d[0]: d[1] for d in data}
df['rating_num0'] = df['rating0'].map(grade_num_map)

In [6]:
df['location0'] = df['location'].apply(lambda x: x[0])

In [7]:
conditions_t = [
    (df['type'] == 'Sport') | (df['type'] == 'Sport, Aid'),
    (df['type'] == 'Boulder'),
    (df['type'] == 'Trad')]
choices_t = ['Sport', 'Boulder', 'Trad']
df['type0'] = np.select(conditions_t, choices_t, default='Trad')

In [8]:
conditions_p = [(df['pitches'] == 1) | (df['pitches'] == '')]
choices_p = ['single-pitch']
df['pitches0'] = np.select(conditions_p, choices_p, default='multi-pitch')

In [9]:
df_features = df[['id','name','type0','rating0','rating_num0','pitches0','location0']]

In [10]:
df_features.shape

(86898, 7)

In [11]:
df_features.drop_duplicates(inplace=True)
df_features.shape

/Users/colinbrochard/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(86892, 7)

In [14]:
df_features.to_csv(write_loc, sep='|')

In [13]:
df_features['id'].value_counts()

107612159    1
108473698    1
107756050    1
108180587    1
106391573    1
111962648    1
114290719    1
105804315    1
105794076    1
107900293    1
108097799    1
105759213    1
113943454    1
107330082    1
105886243    1
106841502    1
106922533    1
106928678    1
112484202    1
109701672    1
105904158    1
114067165    1
112199566    1
105957900    1
105724402    1
106079337    1
105976311    1
107309560    1
108880377    1
106119677    1
105730559    1
106736128    1
113418018    1
105822722    1
112059154    1
105894096    1
113414663    1
106359304    1
111206921    1
106625546    1
106099211    1
112468260    1
107082283    1
107332141    1
107708237    1
112450809    1
111546967    1
107113048    1
107056264    1
108034650    1
106407974    1
105794140    1
113394269    1
105929310    1
105753184    1
105757282    1
108376675    1
108239462    1
108106343    1
106556008    1
107739754    1
106033771    1
107874900    1
111813201    1
112450094    1
105917008    1
108210736 